In [1]:
import tensorflow.lite as tflite
import tensorflow as tf 
from tensorflow import keras 
from io import BytesIO
from urllib import request
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
model = keras.models.load_model('dogs_cats_10_0.687.h5') 

2021-11-28 20:19:02.736653: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
converter = tflite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

2021-11-28 20:19:03.301654: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/41/hnfr5yxd35bg5yjh4tm994980000gp/T/tmpgk_s66kh/assets


2021-11-28 20:19:05.048197: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2021-11-28 20:19:05.048230: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2021-11-28 20:19:05.049311: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/41/hnfr5yxd35bg5yjh4tm994980000gp/T/tmpgk_s66kh
2021-11-28 20:19:05.051129: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2021-11-28 20:19:05.051144: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /var/folders/41/hnfr5yxd35bg5yjh4tm994980000gp/T/tmpgk_s66kh
2021-11-28 20:19:05.058958: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2021-11-28 20:19:05.181417: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /var/folders/41/hnfr5yxd35bg5yjh4tm994980000gp/T/tmpgk_s66kh
2021-11

In [4]:
with tf.io.gfile.GFile('dogs_cats.tflite','wb') as f:
    f.write(tflite_model)

Size : 42.8 MB 

In [5]:
interpreter = tf.lite.Interpreter(model_path='dogs_cats.tflite')
interpreter.allocate_tensors() 
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
input_index,output_index

(0, 13)

In [6]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [9]:
img = prepare_image(download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'),(150, 150))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
datagen = ImageDataGenerator(rescale = 1./255)
X = datagen.flow(x, batch_size=1)

In [18]:
X[0].shape

(1, 150, 150, 3)

In [22]:
interpreter.set_tensor(input_index, X[0]) 
interpreter.invoke() 
preds = interpreter.get_tensor(output_index) 
preds

array([[0.770491]], dtype=float32)

In [23]:
model.predict(X)

array([[0.77049136]], dtype=float32)

In [25]:
from keras_image_helper import create_preprocessor
preprocessor = create_preprocessor('xception', target_size=(150, 150))
x = preprocessor.from_url('https://upload.wikimedia.org/wikipedia/commons/1/18/Vombatus_ursinus_-Maria_Island_National_Park.jpg')
interpreter.set_tensor(input_index, x) 
interpreter.invoke() 
preds = interpreter.get_tensor(output_index) 
preds

array([[0.72104645]], dtype=float32)